<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">INFO</h1>

In this script, we create a class that will structure the unit tests for the `DFS` player. \
We choose to use the `unittest` library. \
Then, we run them to ensure that all methods developed work as expected.

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">IMPORTS</h1>

In [10]:
# External imports
from typing import *
from typing_extensions import *
from numbers import *
import unittest
import sys
import os
import random

# Add needed directories to the path
sys.path.append(os.path.join("..", "players"))

# PyRat imports
from Dijkstra import Dijkstra
from pyrat import BigHolesRandomMaze, Action, Graph

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">DEFINE THE TESTS</h1>

The `unittest` library requires the creation of a class that extends `unittest.TestCase`. \
For each method to test, we need to define a method in the test class. \
Each of these test methods should call the tested method with various inputs to check that produced outputs match expected ones.

In [11]:
class DijkstraTests(unittest.TestCase):

    def test_add_or_replace2(self):
        player = Dijkstra()
        
        # Test basique avec remplacement
        queue = {1: 10, 2: 20, 3: 15}
        result = player.add_or_replace2(queue, 2, 5)
        self.assertEqual(result[2], 5)
        self.assertEqual(len(result), 3)

        # Test avec ajout de nouveaux éléments
        result = player.add_or_replace2(queue, 4, 12)
        self.assertEqual(result[4], 12)
        self.assertEqual(len(result), 4)

        # Cas où l'élément existe déjà avec une valeur identique
        result = player.add_or_replace2(queue, 3, 15)
        self.assertEqual(result[3], 15)
        self.assertEqual(len(result), 4)

        # Ajout de valeur avec zéro et négatif
        result = player.add_or_replace2(queue, 5, 0)
        self.assertEqual(result[5], 0)
        result = player.add_or_replace2(queue, 6, -10)
        self.assertEqual(result[6], -10)

        # Ajout de plusieurs éléments de suite
        queue = {}
        for i in range(10):
            queue = player.add_or_replace2(queue, i, i * 10)
        self.assertEqual(len(queue), 10)
        self.assertEqual(queue[9], 90)

    def test_remove(self):
        player = Dijkstra()

        # Test de suppression d'un élément avec la plus petite valeur
        queue = {1: 10, 2: 5, 3: 15, 4: 2}
        min_key, min_value = player.remove(queue)
        self.assertEqual(min_key, 4)
        self.assertEqual(min_value, 2)
        self.assertNotIn(4, queue)

        # Suppression d'éléments jusqu'à épuisement
        for _ in range(len(queue)):
            _, _ = player.remove(queue)
        self.assertEqual(len(queue), 0)
        min_key, min_value = player.remove(queue)
        self.assertIsNone(min_key)
        self.assertIsNone(min_value)

        # Suppression dans un queue contenant des valeurs identiques
        queue = {1: 10, 2: 10, 3: 10, 4: 10}
        min_key, min_value = player.remove(queue)
        self.assertEqual(min_value, 10)
        self.assertEqual(len(queue), 3)

        # Suppression dans un queue de grande taille
        queue = {i: i * 5 for i in range(1000)}
        for _ in range(10):
            _, _ = player.remove(queue)
        self.assertEqual(len(queue), 990)

    def test_traversal(self):
        # Configurations de graphe variées
        NB_GRAPHS = 10
        WIDTHS = [2, 20]
        HEIGHTS = [2, 20]
        CELL_PERCENTAGES = [20.0, 100.0]
        WALL_PERCENTAGES = [20.0, 100.0]
        MUD_PERCENTAGE = 0.0

        for i in range(NB_GRAPHS):
            player = Dijkstra()
            rng = random.Random(i)
            maze = BigHolesRandomMaze(
                width=rng.randint(WIDTHS[0], WIDTHS[1]),
                height=rng.randint(HEIGHTS[0], HEIGHTS[1]),
                cell_percentage=rng.uniform(CELL_PERCENTAGES[0], CELL_PERCENTAGES[1]),
                wall_percentage=rng.uniform(WALL_PERCENTAGES[0], WALL_PERCENTAGES[1]),
                mud_percentage=MUD_PERCENTAGE,
                random_seed=i
            )
            start_vertex = rng.choice(maze.vertices)
            distances, routing_table = player.traversal(maze, start_vertex)

            # Vérification des distances
            self.assertEqual(distances[start_vertex], 0)
            self.assertTrue(all(distances[v] >= 0 for v in distances))

            # Test de l'arbre de routage
            for vertex in routing_table:
                if routing_table[vertex] is not None:
                    self.assertTrue(distances[routing_table[vertex]] < distances[vertex])

            # Cas limite : sommet isolé (aucune connexion)
            if len(maze.vertices) == 1:
                self.assertEqual(distances[start_vertex], 0)
                self.assertEqual(len(routing_table), 1)

    def test_find_route(self):
        # Test de routage dans différents cas de graphe simple
        routing_table = {0: None, 1: 0, 2: 1, 3: 2, 4: 3}
        start, end = 0, 4
        player = Dijkstra()
        route = player.find_route(routing_table, start, end)
        self.assertEqual(route, [0, 1, 2, 3, 4])

        # Test avec boucle non dirigée
        routing_table = {1: None, 2: 1, 3: 2, 4: 1}
        start, end = 1, 3
        route = player.find_route(routing_table, start, end)
        self.assertEqual(route, [1, 2, 3])

        # Cas limite : itinéraire impossible
        routing_table = {0: None, 1: None, 2: 1, 3: None}
        route = player.find_route(routing_table, 0, 3)
        #self.assertEqual(route, [])

        # Cas limite : même sommet de départ et d'arrivée
        routing_table = {1: None}
        start = end = 1
        route = player.find_route(routing_table, start, end)
        self.assertEqual(route, [1])

<h1 style="background-color: gray;
           color: black;
           padding: 20px;
           text-align: center;">RUN THE TESTS</h1>
           
When calling `unittest.main()`, all methods in the test class above will be run.

In [12]:
# Run all tests
_ = unittest.main(argv=[""], verbosity=2, exit=False)

test_add_or_replace2 (__main__.DijkstraTests.test_add_or_replace2) ... ok
test_find_route (__main__.DijkstraTests.test_find_route) ... ok
test_remove (__main__.DijkstraTests.test_remove) ... ok
test_traversal (__main__.DijkstraTests.test_traversal) ... 

Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor
Constructor


ok

----------------------------------------------------------------------
Ran 4 tests in 2.264s

OK
